# **How RayDP works together with Ray**

RayDP is a distributed data processing library that provides simple APIs for running Spark on Ray and integrating Spark with distributed deep learning and machine learning frameworks. This document builds an end-to-end deep learning pipeline on a single Ray cluster by using Spark for data preprocessing, and uses ray train to complete the training and evaluation.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oap-project/raydp/blob/master/tutorials/raytrain_example.ipynb)

## 1. Colab enviroment Setup

RayDP requires Ray and PySpark. At the same time, pytorch is used to build deep learning model.

In [ ]:
! pip install ray==1.9
# install RayDP nightly build
! pip install raydp-nightly
# or use the released version
# ! pip install raydp
! pip install ray[tune]
! pip install torch==1.8.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

Package                       Version
----------------------------- ------------------------------
absl-py                       1.0.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.4
branca                        0.

## 2. Get the data file

The dataset is from: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset, and we store the file in github repository. It's used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient. 

In [ ]:
! wget https://raw.githubusercontent.com/oap-project/raydp/master/tutorials/dataset/healthcare-dataset-stroke-data.csv -O healthcare-dataset-stroke-data.csv

## 3. Init or connect to a ray cluster

In [ ]:
import ray

ray.init(num_cpus=6)

{'metrics_export_port': 44425,
 'node_id': '1e8d9137f4f4a9343edcce64f2d61d72ec70d624899d07e818bd34e1',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2022-05-20_06-51-06_717688_61/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2022-05-20_06-51-06_717688_61/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2022-05-20_06-51-06_717688_61',
 'webui_url': None}

## 4. Get a spark session

In [ ]:
import raydp

app_name = "Stoke Prediction with RayDP"
num_executors = 1
cores_per_executor = 1
memory_per_executor = "500M"
spark = raydp.init_spark(app_name, num_executors, cores_per_executor, memory_per_executor)

(RayDPSparkMaster pid=529) WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.


(RayDPSparkMaster pid=529) WARNING: An illegal reflective access operation has occurred
(RayDPSparkMaster pid=529) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/spark-unsafe_2.12-3.2.1.jar) to constructor java.nio.DirectByteBuffer(long,int)
(RayDPSparkMaster pid=529) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(RayDPSparkMaster pid=529) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(RayDPSparkMaster pid=529) WARNING: All illegal access operations will be denied in a future release


(RayDPSparkMaster pid=529) 2022-05-20 06:51:17,557 WARN NativeCodeLoader [Thread-2]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
(RayDPSparkMaster pid=529) 2022-05-20 06:51:17,872 INFO SecurityManager [Thread-2]: Changing view acls to: root
(RayDPSparkMaster pid=529) 2022-05-20 06:51:17,879 INFO SecurityManager [Thread-2]: Changing modify acls to: root
(RayDPSparkMaster pid=529) 2022-05-20 06:51:17,880 INFO SecurityManager [Thread-2]: Changing view acls groups to: 
(RayDPSparkMaster pid=529) 2022-05-20 06:51:17,882 INFO SecurityManager [Thread-2]: Changing modify acls groups to: 
(RayDPSparkMaster pid=529) 2022-05-20 06:51:17,883 INFO SecurityManager [Thread-2]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(root); groups with view permissions: Set(); users  with modify permissions: Set(root); groups with modify permissions: Set()
(RayDPSparkMaster pid=529) 2022-05-20 06:51:19,091 I

## 5. Get data from .csv file via 'spark' created by **raydp**

In [ ]:
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load("/content/healthcare-dataset-stroke-data.csv")

(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/spark-unsafe_2.12-3.2.1.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(raylet) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(raylet) WARNING: All illegal access operations will be denied in a future release


## 6. Define the data_process function

The dataset is converted to `pyspark.sql.dataframe.DataFrame`. Before feeding into the deep learning model, we can use raydp to do some transformation operations on dataset.

### 6.1 Data Analysis

Here is a part of the data analysis.

In [ ]:
# Data overview
data.show(5)
# Statistical N/A distribution
# There are 201 'N/A' value in column 'bmi column',
# we can update them the mean of the column
data.describe().show()
data.filter(data.bmi=='N/A').count()
# Observe the distribution of the column 'gender'
# Then we should remove the outliers 'Other'
data.rollup(data.gender).count().show()
# Observe the proportion of positive and negative samples.
data.rollup(data.stroke).count().show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

### 6.2 Define operations

Define data processing operations based on data analysis results.

In [ ]:
from pyspark.sql.functions import hour, quarter, month, year, dayofweek, dayofmonth, weekofyear, col, lit, udf, abs as functions_abs, avg

In [ ]:
# Delete the useless column 'id'
def drop_col(data):
    data = data.drop('id')
    return data

In [ ]:
# Replace the value N/A in 'bmi'
def replace_nan(data):
    bmi_avg = data.agg(avg(col("bmi"))).head()[0]

    @udf("float")
    def replace_nan(value):
        if value=='N/A':
            return float(bmi_avg)
        else:
            return float(value)

    # Replace the value N/A
    data = data.withColumn('bmi', replace_nan(col("bmi")))
    return data

In [ ]:
# Drop the only one value 'Other' in column 'gender'
def clean_value(data):
    data = data.filter(data.gender != 'Other')
    return data

In [ ]:
# Transform the category columns
def trans_category(data):
    @udf("int")
    def trans_gender(value):
        gender = {'Female': 0,
                  'Male': 1}
        return int(gender[value])

    @udf("int")
    def trans_ever_married(value):
        residence_type = {'No': 0,
                          'Yes': 1}
        return int(residence_type[value])

    @udf("int")
    def trans_work_type(value):
        work_type = {'children': 0,
                     'Govt_job': 1,
                     'Never_worked': 2,
                     'Private': 3,
                     'Self-employed': 4}
        return int(work_type[value])

    @udf("int")
    def trans_residence_type(value):
        residence_type = {'Rural': 0,
                          'Urban': 1}
        return int(residence_type[value])

    @udf("int")
    def trans_smoking_status(value):
        smoking_status = {'formerly smoked': 0,
                          'never smoked': 1,
                          'smokes': 2,
                          'Unknown': 3}
        return int(smoking_status[value])

    data = data.withColumn('gender', trans_gender(col('gender'))) \
               .withColumn('ever_married', trans_ever_married(col('ever_married'))) \
               .withColumn('work_type', trans_work_type(col('work_type'))) \
               .withColumn('Residence_type', trans_residence_type(col('Residence_type'))) \
               .withColumn('smoking_status', trans_smoking_status(col('smoking_status')))
    return data

In [ ]:
# Add the discretized column of 'Age'
def map_age(data):
    @udf("int")
    def get_value(value):
        if value >= 18 and value < 26:
            return int(0)
        elif value >=26 and value < 36:
            return int(1)
        elif value >=36 and value < 46:
            return int(2)
        elif value >=46 and value < 56:
            return int(3)
        else:
            return int(4)

    data = data.withColumn('age_dis', get_value(col('age')))
    return data

In [ ]:
# Preprocess the data
def data_preprocess(data):
    data = drop_col(data)
    data = replace_nan(data)
    data = clean_value(data)
    data = trans_category(data)
    data = map_age(data)
    return data

## 7. Data processing

In [ ]:
import torch
from raydp.utils import random_split

# Transform the dataset
data = data_preprocess(data)
# Split data into train_dataset and test_dataset
train_df, test_df = random_split(data, [0.8, 0.2], 0)
# Balance the positive and negative samples
train_df_neg = train_df.filter(train_df.stroke == '1')
train_df = train_df.unionByName(train_df_neg)
train_df = train_df.unionByName(train_df_neg)
features = [field.name for field in list(train_df.schema) if field.name != "stroke"]
# Convert spark dataframe into ray Dataset
# Remember to align ``parallelism`` with ``num_workers`` of ray train
train_dataset = ray.data.from_spark(train_df, parallelism = 8)
test_dataset = ray.data.from_spark(test_df, parallelism = 8)
feature_dtype = [torch.float] * len(features)

## 8. Define a neural network model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class NET_Model(nn.Module):
    def __init__(self, cols):
        super().__init__()
        self.emb_layer_gender = nn.Embedding(2, 2)           # gender
        self.emb_layer_hypertension = nn.Embedding(2,2)      # hypertension
        self.emb_layer_heart_disease = nn.Embedding(2,2)     # heart_disease
        self.emb_layer_ever_married = nn.Embedding(2, 2)     # ever_married
        self.emb_layer_work = nn.Embedding(5, 5)             # work_type
        self.emb_layer_residence = nn.Embedding(2, 2)        # Residence_type
        self.emb_layer_smoking_status = nn.Embedding(4, 4)   # smoking_status
        self.emb_layer_age = nn.Embedding(5, 5)              # age column after discretization
        self.fc1 = nn.Linear(cols, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 2)

        self.fc_sparse = nn.Linear(24, 16)
        self.fc_dense = nn.Linear(3, 8)
        self.fc = nn.Linear(24, 2)
        
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(16)

    def forward(self, *x):
        x = torch.cat(x, dim=1)
        # pick the dense attribute columns
        dense_columns = x[:, [1,7,8]]
        # Embedding operation on sparse attribute columns
        sparse_col_1 = self.emb_layer_gender(x[:, 0].long())
        sparse_col_2 = self.emb_layer_hypertension(x[:, 2].long())
        sparse_col_3 = self.emb_layer_heart_disease(x[:, 3].long())
        sparse_col_4 = self.emb_layer_ever_married(x[:, 4].long())
        sparse_col_5 = self.emb_layer_work(x[:, 5].long())
        sparse_col_6 = self.emb_layer_residence(x[:, 6].long())
        sparse_col_7 = self.emb_layer_smoking_status(x[:, 9].long())
        sparse_col_8 = self.emb_layer_age(x[:, 10].long())
        # Splice sparse attribute columns and dense attribute columns
        x = torch.cat([dense_columns, sparse_col_1, sparse_col_2, sparse_col_3, sparse_col_4, sparse_col_5, sparse_col_6, sparse_col_7, sparse_col_8], dim=1)

        sparse_columns = torch.cat([sparse_col_1, sparse_col_2, sparse_col_3, sparse_col_4, sparse_col_5, sparse_col_6, sparse_col_7, sparse_col_8], dim=1)
        dense_feat = self.fc_dense(dense_columns)
        sparse_feat = self.fc_sparse(sparse_columns)
        return self.fc(torch.cat([dense_feat, sparse_feat], dim=1))

        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = self.fc5(x)
        return x


## 9. Define train and test function

In [ ]:
def train_epoch(dataset, model, criterion, optimizer):
    model.train()
    train_loss, correct, data_size, batch_idx = 0, 0, 0, 0
    for batch_idx, (inputs, targets) in enumerate(dataset):
        # Compute prediction error
        inputs = [inputs[:,i].unsqueeze(1) for i in range(inputs.size(1))]
        targets = targets.reshape(-1)
        outputs = model(*inputs)
        loss = criterion(outputs, targets)
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        data_size += targets.size(0)
        correct += (predicted == targets).sum().item()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Caculate the train_loss and train_acc
    train_loss /= (batch_idx + 1)
    train_acc = correct/data_size
    return train_acc, train_loss

def test_epoch(dataset, model, criterion):
    model.eval()
    test_loss, correct, data_size, batch_idx = 0, 0, 0, 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataset):
            # Compute prediction error
            inputs = [inputs[:,i].unsqueeze(1) for i in range(inputs.size(1))]
            targets = targets.reshape(-1)
            outputs = model(*inputs)
            test_loss += criterion(outputs, targets).item()
            _, predicted = torch.max(outputs.data, 1)
            data_size += targets.size(0)
            correct += (predicted == targets).sum().item()
    # Caculate the test_loss and test_acc
    test_loss /= (batch_idx + 1)
    test_acc = correct/data_size
    return test_acc, test_loss

## 10. Define train function

In [ ]:
from ray import train
from ray.train import get_dataset_shard

def train_func(config):
    num_epochs = config["num_epochs"]
    lr = config["lr"]
    batch_size = config["batch_size"]
    # Then convert to torch datasets
    # Get the corresponging shard
    train_data_shard = get_dataset_shard("train")
    train_dataset = train_data_shard.to_torch(feature_columns=features,
                                              label_column="stroke",
                                              label_column_dtype=torch.long,
                                              feature_column_dtypes=feature_dtype,
                                              batch_size=batch_size)
    test_data_shard = get_dataset_shard("test")
    test_dataset = test_data_shard.to_torch(feature_columns=features,
                                            label_column="stroke",
                                            label_column_dtype=torch.long,
                                            feature_column_dtypes=feature_dtype,
                                            batch_size=batch_size)
    model = NET_Model(len(features))
    model = train.torch.prepare_model(model)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.35, 0.65]))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_results = []
    for epoch in range(num_epochs):
        train_acc, train_loss = train_epoch(train_dataset, model, criterion, optimizer)
        test_acc, test_loss = test_epoch(test_dataset, model, criterion)
        train.report(epoch = epoch, train_acc = train_acc, train_loss = train_loss)
        train.report(epoch = epoch, test_acc=test_acc, test_loss=test_loss)
        loss_results.append(test_loss)

## 11. Define the callback function

In [ ]:
from ray.train import TrainingCallback
from typing import List, Dict

# log the train results
class PrintingCallback(TrainingCallback):
    def handle_result(self, results: List[Dict], **info):
        print(results)

## 12. Train model via ray train

In [ ]:
from ray.train import Trainer

trainer = Trainer(backend="torch", num_workers=num_executors)
trainer.start()
results = trainer.run(
    train_func, config={"num_epochs": 10, "lr": 0.001, "batch_size": 64},
    callbacks=[PrintingCallback()],
    dataset={
        "train": train_dataset,
        "test": test_dataset
    }
)
trainer.shutdown()

2022-05-20 06:52:09,475	INFO trainer.py:172 -- Trainer logs will be logged in: /root/ray_results/train_2022-05-20_06-52-09
2022-05-20 06:52:10,966	INFO trainer.py:178 -- Run results will be logged in: /root/ray_results/train_2022-05-20_06-52-09/run_001
(BaseWorkerMixin pid=1234) 2022-05-20 06:52:10,963	INFO torch.py:67 -- Setting up process group for: env:// [rank=0, world_size=1]
(BaseWorkerMixin pid=1234) 2022-05-20 06:52:11,020	INFO torch.py:239 -- Moving model to device: cpu


[{'epoch': 0, 'train_acc': 0.6178715761113606, 'train_loss': 2.4214471597756657, '_timestamp': 1653029531, '_time_this_iter_s': 0.8797011375427246, '_training_iteration': 1}]
[{'epoch': 0, 'test_acc': 0.7587844254510921, 'test_loss': 0.7357403390547809, '_timestamp': 1653029531, '_time_this_iter_s': 0.0008981227874755859, '_training_iteration': 2}]
[{'epoch': 1, 'train_acc': 0.797485406376291, 'train_loss': 0.6913646668195724, '_timestamp': 1653029532, '_time_this_iter_s': 0.5976800918579102, '_training_iteration': 3}]
[{'epoch': 1, 'test_acc': 0.9021842355175689, 'test_loss': 0.31034330848385305, '_timestamp': 1653029532, '_time_this_iter_s': 0.0007421970367431641, '_training_iteration': 4}]
[{'epoch': 2, 'train_acc': 0.8432869330938483, 'train_loss': 0.4305501192808151, '_timestamp': 1653029533, '_time_this_iter_s': 0.5868339538574219, '_training_iteration': 5}]
[{'epoch': 2, 'test_acc': 0.9069325735992403, 'test_loss': 0.2829096720499151, '_timestamp': 1653029533, '_time_this_iter_s

## 13. shut down ray and raydp

In [ ]:
raydp.stop_spark()
ray.shutdown()